<a href="https://colab.research.google.com/github/jpbeaud/dbscan/blob/main/client.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import time

def string_to_number(s):
  return int(''.join(str(ord(char)) for char in s))



chemin = "/content/sample_data/"
debut = time.time()
print("Début à", debut)



fichier_DH = pd.read_csv(chemin+"extrac_DH.csv", sep=",", dtype=str,skipinitialspace=True)

fichier_DH = fichier_DH.replace("", np.nan)
fichier_DH = fichier_DH.fillna(0)

fichier_DH = fichier_DH.astype(str)
col_ref = ['id_personne']
references_DH = fichier_DH[col_ref]



fichier_DH.sort_values(by='id_personne')

fichier_DH2 = fichier_DH
fichier_DH2 = fichier_DH2.drop('id_personne', axis='columns')

fichier_DH2= fichier_DH2.astype(str)
fichier_DH2 = fichier_DH2.map(string_to_number)



#fichier_DH.set_index('id_personne', inplace=True) # inplace=True --> on ne crée pas un nouveau DF




fichier_SIO = pd.read_csv(chemin+"extrac_SIO.csv", sep=",", dtype=str,skipinitialspace=True)
col_ref = ['client_reference']

references_SIO= fichier_SIO[col_ref]
ref_cli = references_SIO.to_numpy()

fichier_SIO = fichier_SIO.replace("", np.nan)
fichier_SIO = fichier_SIO.fillna(0)



fichier_SIO.sort_values(by='client_reference')
fichier_SIO = fichier_SIO.astype(str)
fichier_SIO = fichier_SIO.map(string_to_number)


fichier_SIO = fichier_SIO.drop('client_reference',axis='columns')
#print ("fichier SIO format :")
#print(fichier_SIO.head())
# X = fichier_DH.to_numpy()



Début à 1730983408.8803294


Ficher DB2 et SIO

In [3]:
# prompt: comment calculer le nombre d'enregistrements différents dans fichier_DH2

# Calculate the number of unique records in fichier_DH2
unique_records_count = len(fichier_DH2.drop_duplicates())

print(f"Number of unique records in fichier_DH2: {unique_records_count}")

# Calculate the number of unique records in fichier_SIO
unique_records_count = len(fichier_SIO.drop_duplicates())

print(f"Number of unique records in fichier_SIO: {unique_records_count}")

Number of unique records in fichier_DH2: 14725
Number of unique records in fichier_SIO: 17079


Appel à DBSCAN

In [4]:
X = fichier_DH2
#X.set_index('id_personne', inplace=True) # inplace=True --> on ne crée pas un nouveau DF

# print(X.dtype)
X = X.apply(pd.to_numeric)

import matplotlib.pyplot as plt
# pltcatter(X[:, 3], X[:, 4])


#plt.scatter(x="pcs" , y="cosop", c = 'r', data=X)
#plt.show()

from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler

# création du jeu de test

#X = StandardScaler().fit_transform(X)
#print ("fichier DH2 après scaler :")
#print(X[:5])

from sklearn import metrics
from sklearn.cluster import DBSCAN
from sklearn.cluster import OPTICS
min_groupe = 5
db = DBSCAN(eps=0.000000000000001,min_samples=min_groupe).fit(X)
labels_DH = db.labels_

n_clusters_ = len(set(labels_DH)) - (1 if -1 in labels_DH else 0)
n_noise_ = list(labels_DH).count(-1)

print("Estimated number of clusters DH: %d" % n_clusters_)
print("Estimated number of noise points DH: %d" % n_noise_)


Y = fichier_SIO


#Y = StandardScaler().fit_transform(Y)
#print ("fichier SIO après scaler :")
#print(Y[:5])


# Y = Y.apply(pd.to_numeric)
db2 = DBSCAN(eps=0.00000000000001, min_samples=min_groupe).fit(Y)
labels_SIO = db2.labels_



n_clusters_ = len(set(labels_SIO)) - (1 if -1 in labels_SIO else 0)
n_noise_ = list(labels_SIO).count(-1)

print("Estimated number of clusters SIO: %d" % n_clusters_)
print("Estimated number of noise points SIO: %d" % n_noise_)
print("____________Résumé____________")
difference = labels_DH - labels_SIO
print("Données en écarts = ", difference)
print(labels_DH)
print(labels_SIO)
n_DH = len(labels_DH)
n_SIO=len(labels_SIO)
n_dif = len(difference)
print("Longueur dif = ", n_dif)
fin = False
i=-1

nb_ko = 0

# Créer un tableau vide avec une dimension initiale
empty_array = np.empty((0, 3))  # Par exemple, un tableau vide avec 3 colonnes



Estimated number of clusters DH: 1855
Estimated number of noise points DH: 16678
Estimated number of clusters SIO: 1844
Estimated number of noise points SIO: 19101
____________Résumé____________
Données en écarts =  [  0   0   0 ... -10  12   0]
[  0  -1   1 ...   4 727  -1]
[  0  -1   1 ...  14 715  -1]
Longueur dif =  100000


On sélectionne les enregistrements qui ont 0 comme label

In [5]:
labels_DH_zero = [x for x in labels_DH if x ==0]
print("longueur DH label 0 =", len(labels_DH_zero))
labels_SIO_zero = [x for x in labels_SIO if x ==0]
print("longueur SIO label 0 =", len(labels_SIO_zero))


longueur DH label 0 = 203
longueur SIO label 0 = 190


In [6]:
# prompt: transform labels_DH into a dataframe

#print("DH.38=", X[38], "SIO.38=", Y[38])
#print("DH.0=", X[0], "SIO.0=", Y[0])

# Assuming 'labels_DH' is already defined from the previous code
labels_DH_df = pd.DataFrame({'labels_DH': labels_DH})
labels_SIO_df = pd.DataFrame({'labels_SIO': labels_SIO})

labels_DH_0 = labels_DH_df[labels_DH_df['labels_DH'] == 0]
print(labels_DH_0.head())
labels_SIO_0 = labels_SIO_df[labels_SIO_df['labels_SIO'] == 0]
print(labels_SIO_0.head())
print("______________________________________________________")
#print(labels_DH[38], " SIO=", labels_SIO[38])
#print("Référence SIO", references_SIO.iloc[38], ". Référence DH = ", references_DH.iloc[38])




    labels_DH
0           0
12          0
29          0
40          0
53          0
    labels_SIO
0            0
12           0
29           0
40           0
53           0
______________________________________________________


Extraction des index en décalage

In [7]:
# prompt: extraire les index qui dont dans labels_DH_0 mais pas dans labels_SIO_0

# Find indices present in labels_DH_0 but not in labels_SIO_0
dh_indices_in_0 = labels_DH_0.index.tolist()
sio_indices_in_0 = labels_SIO_0.index.tolist()

indices_in_sio_not_dh = list(set(sio_indices_in_0) - set(dh_indices_in_0))

print("Indices in labels_DH_0 but not in labels_SIO_0:")
print(indices_in_sio_not_dh)
len(indices_in_sio_not_dh)

Indices in labels_DH_0 but not in labels_SIO_0:
[137, 12243]


2

Test A

In [13]:
print(sio_indices_in_0)
print("Label DH SIO du 0ème ", labels_DH[0])
print("Label DH du 12 ème ",labels_DH[12])
print("Label DH du 1er enr. = ", labels_DH[sio_indices_in_0[0]], "pos=", sio_indices_in_0[0])
print("Label SIO du 2èmz enr. = ", labels_SIO[sio_indices_in_0[1]], "pos=", sio_indices_in_0[1])

[0, 12, 29, 40, 53, 57, 64, 73, 75, 101, 105, 107, 110, 123, 124, 137, 148, 150, 178, 196, 269, 271, 278, 294, 306, 396, 574, 689, 703, 749, 785, 961, 1289, 1635, 2705, 4650, 9561, 10024, 10162, 10243, 10538, 10572, 10583, 10744, 11464, 11640, 11716, 12243, 12401, 13531, 14285, 15486, 15850, 17716, 19101, 19422, 19548, 21496, 22740, 23109, 24401, 26017, 27145, 27238, 27527, 27721, 27752, 28062, 28102, 29527, 30933, 31471, 33326, 33977, 34810, 36023, 36162, 37319, 37349, 38292, 38453, 38460, 38496, 38498, 38500, 38505, 38544, 38556, 38561, 38572, 38573, 38579, 38601, 38837, 39017, 39507, 39614, 39674, 40082, 40093, 41678, 42765, 43269, 46354, 46467, 46610, 46611, 47420, 47470, 47909, 47927, 48102, 48373, 48395, 49050, 50502, 51369, 51603, 52052, 52535, 54472, 55172, 58810, 59223, 59579, 61616, 61767, 62638, 63352, 64945, 65232, 65911, 66220, 67926, 72680, 73311, 75921, 75930, 76087, 76093, 76112, 76134, 76138, 76150, 76162, 76168, 76197, 76201, 76205, 76209, 76222, 76232, 76376, 76438, 

In [14]:
#i0 = indices_in_sio_not_dh[0]
#print("Donnée DH avec indice ", i0)
#print(X.iloc[i0])
#print("Donnée SIO avec indice ", i0)
#print(Y.iloc[i0])
#print("Référence SIO", references_SIO.iloc[i0], ". Référence DH = ", references_DH.iloc[i0])
liste_ref_KO = []
for i in range(2):
  labels_SIO_i = labels_SIO_df[labels_SIO_df['labels_SIO'] == i]
  sio_indices_in_i = labels_SIO_i.index.tolist()
  for j in range(len(sio_indices_in_0)):
    k = labels_DH[sio_indices_in_0[j]]
    labels_DH_k = labels_DH_df[labels_DH_df['labels_DH'] == k]
    dh_indices_in_k = labels_DH_k.index.tolist()
  indices_en_decallage = list(set(sio_indices_in_i) - set(dh_indices_in_k))
  if len(indices_en_decallage) > 0:
    print("Indices en décallage:", indices_en_decallage)
    liste_ref_KO.append(references_SIO.iloc[indices_en_decallage])
print("__________________________________________________")
print ("Longueur anomalie=", len(liste_ref_KO))
for j in range(len(liste_ref_KO)):
  print("Réf ", j, " = ", liste_ref_KO[j])



Indices en décallage: [137, 12243]
Indices en décallage: [2, 50980, 52229, 15892, 50996, 25014, 26462]
__________________________________________________
Longueur anomalie= 2
Réf  0  =        client_reference
137         0000019260
12243       0100693500
Réf  1  =        client_reference
2           0000000280
50980       1101286371
52229       1101863671
15892       0102401840
50996       1101293961
25014       0107042750
26462       0108005010


In [ ]:
#i0 = indices_in_sio_not_dh[0]
#print("Donnée DH avec indice ", i0)
#print(X.iloc[i0])
#print("Donnée SIO avec indice ", i0)
#print(Y.iloc[i0])
#print("Référence SIO", references_SIO.iloc[i0], ". Référence DH = ", references_DH.iloc[i0])
liste_ref_KO = []
for i in range(10):
  labels_SIO_i = labels_SIO_df[labels_SIO_df['labels_SIO'] == i]
  sio_indices_in_i = labels_SIO_i.index.tolist()
  for j in range(len(sio_indices_in_0)):
    k = labels_DH[sio_indices_in_0[j]]
    dh_indices_in_i = labels_DH_i.index.tolist()
  k = labels_DH[sio_indices_in_0[0]]
  dh_indices_in_i = labels_DH_i.index.tolist()

  indices_in_sio_not_dh_i = list(set(sio_indices_in_i) - set(dh_indices_in_i))
  if len(indices_in_sio_not_dh_i) > 0:
    print("Indices in labels_DH_i but not in labels_SIO_i:", indices_in_sio_not_dh_i)
    liste_ref_KO.append(references_SIO.iloc[indices_in_sio_not_dh_i])
print("__________________________________________________")
print ("Longueur anomalie=", len(liste_ref_KO))
for j in range(len(liste_ref_KO)):
  print("Réf ", j, " = ", liste_ref_KO[j])



Indices in labels_DH_i but not in labels_SIO_i: [137, 12243]
Indices in labels_DH_i but not in labels_SIO_i: [94016, 67]
Indices in labels_DH_i but not in labels_SIO_i: [12019]
Indices in labels_DH_i but not in labels_SIO_i: [61954, 86025, 16, 18960, 53784, 56346, 61982, 92191, 55842, 51239, 55849, 32299, 31291, 12349, 89159, 38984, 38481, 49747, 98391, 56929, 40040, 87166, 95369, 30346, 67725, 27798, 17570, 33956, 87207, 16555, 87214, 71861, 89269, 50360, 24763, 189, 45245, 85703, 208, 38612, 19158, 52438, 36573, 55008, 225, 66788, 10983, 57584, 14068, 19700, 54021, 47891, 16661, 19736, 50968, 88345, 14110, 287, 90919, 11049, 56620, 70956, 19250, 20275, 59702, 69945, 86849, 95042, 11588, 17735, 66892, 57181, 76638, 2912, 23412, 52598, 49528, 94594, 30598, 46470, 53127, 86410, 20883, 91541, 68504, 27034, 17827, 25516, 76727, 92600, 99768, 14267, 26044, 40389, 50122, 50640, 11217, 978, 48083, 17878, 95727, 66547, 60404, 66042, 21501]
Indices in labels_DH_i but not in labels_SIO_i: [5785

In [ ]:
# Deux tableaux à une dimension
#array1 = np.array([1, 2, 3])
#array2 = np.array([4, 5, 6]) # Empiler verticalement
#result = np.vstack((array1, array2))
#print(result)
#print(array1.shape)

#print(result.shape)


liste = []
A = np.empty((0, 10000), dtype=int)

for i in range(10000):
  liste = []
  for j in range(10000):
    liste.append(labels_DH[i] -labels_DH[j])
  A = np.vstack([A, liste])


print(A)



KeyboardInterrupt: 

Fin

In [ ]:
RRR






while (i < n_dif - 1):
    i = i + 1
    if (difference[i] != 0):
        print("i=",i)
        print("différence enregistrement ", str(i), " DH=", labels_DH[i], "// SIO=",labels_SIO[i], "// ref=", ref_cli[i,:])
        #print("enr DH=", fichier_DH.loc[i,:])
        #print("enr SIO=",fichier_SIO.loc[i,:])
        nb_ko = nb_ko + 1
        X = np.delete(X,i,0)
        Y = np.delete(Y,i,0)
        ref_cli = np.delete(ref_cli,i,0)
        db = DBSCAN(eps=0.000000000000001,min_samples=min_groupe).fit(X)
        labels_DH = db.labels_
        db2 = DBSCAN(eps=0.00000000000001, min_samples=min_groupe).fit(Y)
        labels_SIO = db2.labels_
        difference = labels_DH - labels_SIO
        n_dif = n_dif - 1


lg = len(difference)
#nb_ano = 0
#for i in range(lg):
#    if difference[i] !=0:
#        nb_ano = nb_ano + 1
print("ANOMALIES = ", nb_ko)
#  3117773703

fin_exec = time.time()
temps = fin_exec - debut
print(f'Temps d\'exécution : {temps:.2}ms')